# Convert Comprehend Medical JSON Outputs to CSV

This project converts outputs from Comprehend Medical Analysis Job into Pandas dataframes. There are four types of AWS Comprehend Medical APIs/types of analyses:
1. Named entity and relationship
2. Protected Health Information
3. ICD-10-CM identification and linking
4. RxNorm identification and linking

To create and run an analysis project in Amazon Comprehend Medical, refer to the guides below.

**Reference**

General: https://aws.amazon.com/comprehend/medical/

Setting up account: https://docs.aws.amazon.com/comprehend/latest/dg/getting-started-med.html

Pricing: https://aws.amazon.com/comprehend/pricing/

Data Guidelines and Quotas: https://docs.aws.amazon.com/comprehend/latest/dg/guidelines-and-limits-med.html

Data protection: https://docs.aws.amazon.com/comprehend/latest/dg/data-protection.html

Access Management and permissions: https://docs.aws.amazon.com/comprehend/latest/dg/access-control-managing-permissions-med.html


In [1]:
import pandas as pd
import json
import os

### Data Processing - Entities

In [2]:
# Load output file from the local server

local_cm_folder = 'Comprehend/outputs/medical/' # local Comprehend Medical folder
project_folder = 'medical_entities_test/' # local project resutls folder
results_filename = 'A Case of 2019 Novel Coronavirus.txt.out' # Output filename

local_results_filename = local_cm_folder+project_folder+results_filename

# Alternatively, load the results file from GitHub
# local_results_filename = 'insert github link here'

In [3]:
# Load one results file

results = [json.loads(line) for line in open(local_results_filename, 'r', encoding = 'utf-8')]
print('Number of records in the output:',len(results[0]['Entities']))

Number of records in the output: 98


**Code that processes JSON into Dataframes**

In [5]:
def extract_traits(results):
    traits_df = pd.DataFrame()
    for record in results[0]['Entities']:
        try: 
            if len(record['Traits'])>0:
                temp_pd = pd.DataFrame(record['Traits'])
                temp_pd['EntityId'] = record['Id']
                traits_df = traits_df.append(temp_pd)
        except:
            pass
    try: 
        traits_df = traits_df.set_index('EntityId')
    except: 
        pass
    return traits_df  

def extract_attributes(results):
    attributes_df = pd.DataFrame()
    for record in results[0]['Entities']:
        try:
            if len(record['Attributes'])>0:
                temp_pd = pd.DataFrame(record['Attributes'])
                temp_pd['EntityId'] = record['Id']
                attributes_df = attributes_df.append(temp_pd)
        except:
            pass
    try: 
        attributes_df = attributes_df.set_index('EntityId')
    except: 
        pass
  
    return attributes_df

def extract_rxnorm(results):
    rxnorm_df = pd.DataFrame()
    for record in results[0]['Entities']:
        try:
            if len(record['RxNormConcepts'])>0:
                temp_pd = pd.DataFrame(record['RxNormConcepts'])
                temp_pd['EntityId'] = record['Id']
                rxnorm_df = rxnorm_df.append(temp_pd)
        except:
            pass
        
    try: 
        rxnorm_df = rxnorm_df.set_index('EntityId')
    except: 
        pass    
    return rxnorm_df

def extract_icd10(results):
    icd10_df = pd.DataFrame()
    for record in results[0]['Entities']:
        try:
            if len(record['ICD10CMConcepts'])>0:
                temp_pd = pd.DataFrame(record['ICD10CMConcepts'])
                temp_pd['EntityId'] = record['Id']
                icd10_df = icd10_df.append(temp_pd)
        except:
            pass
    
    try: 
        icd10_df = icd10_df.set_index('EntityId')
    except: 
        pass    
    return icd10_df

def extract_entities(results):
    entities_df = pd.DataFrame()
    for entity in results[0]['Entities']:
        temp = entity.copy()
        temp.pop('Traits', None)
        temp.pop('Attributes', None)
        temp.pop('RxNormConcepts', None)
        temp.pop('ICD10CMConcepts', None)
        entities_df = entities_df.append(pd.DataFrame(temp, index = [0]))

    try: 
        entities_df = entities_df.rename(columns = {'Id':'EntityId'}).set_index('EntityId')
    except: 
        pass     
    
    return entities_df

In [6]:
extract_entities(results)

,BeginOffset,EndOffset,Score,Text,Category,Type
EntityId,,,,,,
61,3,18,0.974785,2 February 2020,PROTECTED_HEALTH_INFORMATION,DATE
62,22,24,0.972128,28,PROTECTED_HEALTH_INFORMATION,AGE
76,59,67,0.990530,pregnant,MEDICAL_CONDITION,DX_NAME
77,84,89,0.758245,fever,MEDICAL_CONDITION,DX_NAME
63,84,96,0.150570,fever clinic,PROTECTED_HEALTH_INFORMATION,ADDRESS
...,...,...,...,...,...,...
155,4308,4314,0.604536,"12, 15",PROTECTED_HEALTH_INFORMATION,DATE
156,4365,4376,0.684804,infiltrates,MEDICAL_CONDITION,DX_NAME
162,4385,4389,0.431774,lung,ANATOMY,SYSTEM_ORGAN_SITE


In [9]:
extract_traits(results)

,Name,Score
EntityId,,
76,DIAGNOSIS,0.960524
79,SYMPTOM,0.541633
79,DIAGNOSIS,0.443185
81,DIAGNOSIS,0.704319
82,DIAGNOSIS,0.863402
83,DIAGNOSIS,0.865270
136,DIAGNOSIS,0.791274
137,DIAGNOSIS,0.923236
138,DIAGNOSIS,0.975147


### Save DataFrame to Excel

In [10]:
# Export into Excel parameters

local_cm_folder = 'Comprehend/outputs/medical/'
project_folder = 'medical_entities_test/' #local resutls folder
results_filename = 'A Case of 2019 Novel Coronavirus.txt.out'

In [11]:
def json_to_excel(folder, filename):
    # Load the results
    results = [json.loads(line) for line in open(folder+filename, 'r', encoding = 'utf-8')]
    # name and create the processed results folder
    processed_folder = 'processed/'
    # create output folder if it does not exist
    try: 
        os.mkdir(folder+processed_folder) 
    except OSError as error: 
        pass # do nothing

    output_filename = folder+processed_folder+filename[:-8]+'.xlsx' # -8 deletes 2 extensions from filename
    
    # Create a multisheet Excel writer
    writer = pd.ExcelWriter(output_filename, engine='xlsxwriter')
    
    # Save and Excel file with Entities, Traits, and other outputs as tabs
    if len(results[0]['Entities'])>0:
        extract_entities(results).to_excel(writer, sheet_name = 'Entities', encoding = 'utf-8')
        extract_traits(results).to_excel(writer, sheet_name = 'Traits', encoding = 'utf-8')
        # These attriputes might not be present in the results
        attributes = extract_attributes(results) 
        rxnorm = extract_rxnorm(results)
        icd10 = extract_icd10(results)
        if len(rxnorm)>0:
            rxnorm.to_excel(writer, sheet_name = 'RxNorm', encoding = 'utf-8')
        if len(icd10)>0:
            icd10.to_excel(writer, sheet_name = 'ICD10-CM', encoding = 'utf-8')
        if len(attributes)>0:
            attributes.to_excel(writer, sheet_name = 'Attributes', encoding = 'utf-8')
        
    writer.save()
    
    print(len(results[0]['Entities']),'records saved to \n', output_filename)

In [12]:
json_to_excel(local_cm_folder+project_folder,results_filename)

98 records saved to 
 Comprehend/outputs/medical/medical_entities_test/processed/A Case of 2019 Novel Coronavirus.xlsx


### Save DataFrame to CSV

In [13]:
def json_to_csv(folder, filename):
    # Load the results
    results = [json.loads(line) for line in open(folder+filename, 'r', encoding = 'utf-8')]
    # name and create the processed results folder
    processed_folder = 'processed_csv/'
    # create output folder if it does not exist
    try: 
        os.mkdir(folder+processed_folder) 
    except OSError as error: 
        pass # do nothing

    output_filename_entities = folder+processed_folder+filename[:-8]+'_entities.csv' # -8 deletes 2 extensions
    output_filename_attributes = folder+processed_folder+filename[:-8]+'_attributes.csv'
    output_filename_traits = folder+processed_folder+filename[:-8]+'_traits.csv'
    output_filename_rxnorm = folder+processed_folder+filename[:-8]+'_RxNorm.csv'
    output_filename_icd10 = folder+processed_folder+filename[:-8]+'_ICD-10.csv'

    # Create a multisheet Excel writer
    # writer = pd.ExcelWriter(output_filename, engine='xlsxwriter')
    
    # Save and Excel file with Entities, Traits, and other outputs as tabs
    if len(results[0]['Entities'])>0:
        extract_entities(results).to_csv(output_filename_entities, encoding = 'utf-8')
        extract_traits(results).to_csv(output_filename_traits, encoding = 'utf-8')
        # These attriputes might not be present in the results
        attributes = extract_attributes(results) 
        rxnorm = extract_rxnorm(results)
        icd10 = extract_icd10(results)
        if len(rxnorm)>0:
            rxnorm.to_csv(output_filename_rxnorm, encoding = 'utf-8')
        if len(icd10)>0:
            icd10.to_csv(output_filename_icd10, encoding = 'utf-8')
        if len(attributes)>0:
            attributes.to_csv(output_filename_attributes, encoding = 'utf-8')
 
    print(len(results[0]['Entities']),'records saved to \n', folder+processed_folder)

In [14]:
json_to_csv(local_cm_folder+project_folder,results_filename)

98 records saved to 
 Comprehend/outputs/medical/medical_entities_test/processed_csv/


### Run all files in the folder 

In [15]:
local_cm_folder = 'Comprehend/outputs/medical/'
project_folder = 'rxnorm_test/' #local resutls folder

for file in os.listdir(local_cm_folder+project_folder):
    if file[-4:] == '.out': # is a valid AWS output file
        json_to_excel(local_cm_folder+project_folder, file)

9 records saved to 
 Comprehend/outputs/medical/rxnorm_test/processed/A Case of 2019 Novel Coronavirus.xlsx
8 records saved to 
 Comprehend/outputs/medical/rxnorm_test/processed/A Case Report of an Elderly Woman With Mucous Membrane Pemphigoid.xlsx
12 records saved to 
 Comprehend/outputs/medical/rxnorm_test/processed/Case 17-2007 A 25-Year-Old Woman 0.xlsx
26 records saved to 
 Comprehend/outputs/medical/rxnorm_test/processed/Case 17-2007 A 25-Year-Old Woman 1.xlsx
6 records saved to 
 Comprehend/outputs/medical/rxnorm_test/processed/Case 23-2004 A 50-Year-Old Woman 0.xlsx
1 records saved to 
 Comprehend/outputs/medical/rxnorm_test/processed/Case 23-2004 A 50-Year-Old Woman 1.txt.xlsx
1 records saved to 
 Comprehend/outputs/medical/rxnorm_test/processed/Case 23-2004 A 50-Year-Old Woman 1.xlsx
9 records saved to 
 Comprehend/outputs/medical/rxnorm_test/processed/Case 23-2004 A 50-Year-Old Woman 2txt.xlsx


### RxNorm Concepts

In [16]:
project_folder = 'rxnorm_test/' #local analysis resutls folder
local_results_filename = local_cm_folder+project_folder+results_filename

In [17]:
# Load one results file

results = [json.loads(line) for line in open(local_results_filename, 'r', encoding = 'utf-8')]
print('Number of records in the output:',len(results[0]['Entities']))

# Show Entities
extract_entities(results)

Number of records in the output: 9


,Text,Category,Type,Score,BeginOffset,EndOffset
EntityId,,,,,,
0,Arbidol,MEDICATION,BRAND_NAME,0.655011,2358,2365
2,Suzhou Pharmaceutical Factory,MEDICATION,BRAND_NAME,0.223883,2375,2404
6,lopinavir,MEDICATION,GENERIC_NAME,0.974537,2450,2459
7,ritonavir,MEDICATION,GENERIC_NAME,0.936402,2464,2473
12,cefoperazone sodium,MEDICATION,GENERIC_NAME,0.995457,2531,2550
13,sulbactam sodium,MEDICATION,GENERIC_NAME,0.984302,2555,2571
17,human serum albumin,MEDICATION,GENERIC_NAME,0.425588,2627,2646
21,dexamethasone,MEDICATION,GENERIC_NAME,0.999287,2708,2721
22,magnesium sulfate,MEDICATION,GENERIC_NAME,0.997411,2726,2743


In [18]:
extract_attributes(results).loc[6]

,Type,Score,RelationshipScore,Id,BeginOffset,EndOffset,Text,Traits
EntityId,,,,,,,,
6,ROUTE_OR_MODE,0.991703,0.644543,4,2427,2433,orally,[]
6,FORM,0.992166,0.896451,8,2474,2481,tablets,[]
6,DOSAGE,0.856208,0.989020,9,2483,2494,400/100 mg,[]
6,ROUTE_OR_MODE,0.998038,0.992903,10,2508,2514,orally,[]
6,FREQUENCY,0.999190,0.953145,11,2515,2528,every 8 hours,[]


In [19]:
extract_rxnorm(results).loc[6]

,Description,Code,Score
EntityId,,,
6,lopinavir,195088,0.614041
6,lopinavir 200 MG / Ritonavir 50 MG Oral Tablet,597730,0.381698
6,lopinavir 100 MG / Ritonavir 25 MG Oral Tablet,746645,0.218608
6,lopinavir 200 MG / Ritonavir 50 MG Oral Tablet...,847741,0.078047
6,lopinavir 100 MG / Ritonavir 25 MG Oral Tablet...,847745,0.046521


In [20]:
json_to_excel(local_cm_folder+project_folder, results_filename)

9 records saved to 
 Comprehend/outputs/medical/rxnorm_test/processed/A Case of 2019 Novel Coronavirus.xlsx


### ICD-10 Concepts

In [21]:
project_folder = 'icd-10_test/' #local analysis resutls folder
local_results_filename = local_cm_folder+project_folder+results_filename

In [22]:
results = [json.loads(line) for line in open(local_results_filename, 'r', encoding = 'utf-8')]
print('Number of records in the output:',len(results[0]['Entities']))

Number of records in the output: 12


In [23]:
extract_entities(results)

,Text,Category,Type,Score,BeginOffset,EndOffset
EntityId,,,,,,
0,pregnant,MEDICAL_CONDITION,DX_NAME,0.990530,59,67
1,fever,MEDICAL_CONDITION,DX_NAME,0.758245,84,89
3,fever,MEDICAL_CONDITION,DX_NAME,0.993965,144,149
5,fever,MEDICAL_CONDITION,DX_NAME,0.904484,322,327
9,subpleural patchy consolidation,MEDICAL_CONDITION,DX_NAME,0.416429,650,681
11,ground-glass opacities,MEDICAL_CONDITION,DX_NAME,0.440463,698,720
12,preterm birth,MEDICAL_CONDITION,DX_NAME,0.627147,2285,2298
15,ground-glass opacities,MEDICAL_CONDITION,DX_NAME,0.417119,2924,2946
16,atypical pneumonia,MEDICAL_CONDITION,DX_NAME,0.513483,2993,3011


In [24]:
extract_attributes(results)

,Type,Score,RelationshipScore,Id,BeginOffset,EndOffset,Text,Traits
EntityId,,,,,,,,
3,ACUITY,0.728252,0.999872,2,131,143,intermittent,[]
3,ACUITY,0.412754,1.000000,4,154,160,1 week,[]
9,SYSTEM_ORGAN_SITE,0.470504,0.405845,7,576,581,Chest,[]
9,DIRECTION,0.909073,1.000000,8,639,649,left-sided,[]
11,DIRECTION,0.938604,0.999727,10,686,697,right-sided,[]
15,DIRECTION,0.979477,0.859434,14,2905,2914,bilateral,[]
26,SYSTEM_ORGAN_SITE,0.431774,0.999517,27,4385,4389,lung,[]


In [25]:
extract_icd10(results).loc[28]

,Description,Code,Score
EntityId,,,
28,Asymptomatic menopausal state,Z78.0,0.131600
28,Asymptomatic premature menopause,E28.319,0.110610
28,Asymptomatic varicose veins of unspecified low...,I83.90,0.103053
28,Other forms of plague,A20.8,0.097035
28,Asymptomatic neurosyphilis,A52.2,0.090533


### Protected Health Information

In [26]:
project_folder = 'phi_test/' #local analysis resutls folder
local_results_filename = local_cm_folder+project_folder+results_filename

In [27]:
results = [json.loads(line) for line in open(local_cm_folder+project_folder+results_filename, 'r', encoding = 'utf-8')]
entities = results[0][list(results[0].keys())[0]]
print('Number of records in the output:',len(entities))

Number of records in the output: 19


In [28]:
extract_entities(results)

,BeginOffset,EndOffset,Score,Text,Category,Type
EntityId,,,,,,
0,3,18,0.974785,2 February 2020,PROTECTED_HEALTH_INFORMATION,DATE
1,22,24,0.972128,28,PROTECTED_HEALTH_INFORMATION,AGE
2,84,96,0.150570,fever clinic,PROTECTED_HEALTH_INFORMATION,ADDRESS
3,100,125,0.804315,Suzhou Municipal Hospital,PROTECTED_HEALTH_INFORMATION,ADDRESS
4,201,207,0.947584,Suzhou,PROTECTED_HEALTH_INFORMATION,ADDRESS
5,214,221,0.132450,January,PROTECTED_HEALTH_INFORMATION,DATE
6,257,262,0.833486,Wuhan,PROTECTED_HEALTH_INFORMATION,ADDRESS
7,410,424,0.152510,Chinese Center,PROTECTED_HEALTH_INFORMATION,ADDRESS
8,477,501,0.430996,"BioGerm, Shanghai, China",PROTECTED_HEALTH_INFORMATION,ADDRESS


In [29]:
extract_attributes(results)

""
